In [1]:
from time import time
from pyasn1.type import univ, char, tag, namedtype, constraint, namedval
from pyasn1.codec.ber import encoder, decoder
from trustmessages import *
from asn1ate import parser, sema, pyasn1gen
import random
from io import StringIO

In [2]:
SCHEMA = """QTM DEFINITIONS ::= BEGIN
QualitativeValue ::= ENUMERATED {
    very-bad (0), bad (1), neutral (2), good (3), very-good (4)
}

SL ::= SEQUENCE {
    b REAL, d REAL, u REAL
}

END"""

# parse the schema with asn1ate
_parse_tree = parser.parse_asn1(SCHEMA)
_modules = sema.build_semantic_model(_parse_tree)
_clazz_stream = StringIO()
pyasn1gen.generate_pyasn1(_modules[0], _clazz_stream)
_clazz = _clazz_stream.getvalue()
# execute class in this context
exec(_clazz)

In [3]:
r = Rating()
# r["tms"] = univ.ObjectIdentifier((2, 2, 2))
r["source"] = "djelenc@gmail.com"
r["target"] = "david.jelenc@fri.uni-lj.si"
r["service"] = "seller"
r["date"] = time()
r["value"] = encoder.encode(QualitativeValue("very-good"))

decoded, _ = decoder.decode(encoder.encode(r), asn1Spec=Rating())
assert(r.prettyPrint() == decoded.prettyPrint())
v, _ = decoder.decode(decoded["value"], asn1Spec=QualitativeValue())

In [4]:
text = "service = seller AND source = david@fri.si AND (target = balu@fri.si OR target = aleks@fri.si)"

In [5]:
q = trustutils.create_query(text)
s = encoder.encode(q)
float(len(s)) / len(text)

0.8936170212765957

In [6]:
qtm = trustdatabase.QtmDb()
sl = trustdatabase.SLDb()